<a href="https://colab.research.google.com/github/ShimilSBabu/Siamese-Network-for-Face-Recognition/blob/main/siamese_network_fetch_olivetti_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libs

In [2]:
from sklearn.datasets import fetch_olivetti_faces
from numpy.random import RandomState
import numpy as np
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten, Dropout, GlobalAveragePooling2D, Lambda
import tensorflow as tf
import tensorflow.keras.backend as k

# Model Defenition

In [3]:
def create_model():
    inputs = Input((64, 64, 1))
    x = Conv2D(96, (11, 11), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (5, 5), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(384, (3, 3), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    pooledOutput = GlobalAveragePooling2D()(x)
    pooledOutput = Dense(1024)(pooledOutput)
    outputs = Dense(128)(pooledOutput)

    model = Model(inputs, outputs)
    return model

# Feature Extractor

In [4]:
feature_extractor = create_model()
imgA = Input(shape=(64, 64, 1))
imgB = Input(shape=(64, 64, 1))
featA = feature_extractor(imgA)
featB = feature_extractor(imgB)

# Function to Calculate Euclidian Distance

In [5]:
def euclidean_distance(vectors):
    (featA, featB) = vectors
    sum_squared = k.sum(k.square(featA - featB), axis=1, keepdims=True)
    return k.sqrt(k.maximum(sum_squared, k.epsilon()))

distance = Lambda(euclidean_distance)([featA, featB])
outputs = Dense(1, activation="sigmoid")(distance)
model = Model(inputs=[imgA, imgB], outputs=outputs)

# Compiling the Model

In [6]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Importing the Dataset

In [7]:
rng = RandomState(0)
images_dataset, labels_dataset = fetch_olivetti_faces(return_X_y=True, shuffle=True, random_state=rng)
n_samples, n_features = images_dataset.shape

downloading Olivetti faces from https://ndownloader.figshare.com/files/5976027 to /root/scikit_learn_data


# Resizing the Dataset
The fetched dataset *images_dataset* has size 400\*4064. We need to resize it to 400\*64\*64 to give it as input to our model's training function.

In [8]:
faces_resized = []
for num in range(len(images_dataset)):
  faces_resized.append(cv2.resize(images_dataset[num], dsize=(64, 64)))

# Training Pairs Generation

In [9]:
def generate_train_image_pairs(images_dataset, labels_dataset):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels_dataset) if
                                       label == curr_label])
    
    pair_images = []
    pair_labels = []
    for index, image in enumerate(images_dataset):
        pos_indices = label_wise_indices.get(labels_dataset[index])
        pos_image = images_dataset[np.random.choice(pos_indices)]
        
        image = cv2.resize(image, dsize=(64, 64))
        
        pair_images.append((image, pos_image))
        pair_labels.append(1)

        neg_indices = np.where(labels_dataset != labels_dataset[index])
        neg_image = images_dataset[np.random.choice(neg_indices[0])]
        pair_images.append((image, neg_image))
        pair_labels.append(0)
    return np.array(pair_images), np.array(pair_labels)

# Model Training

In [10]:
images_pair, labels_pair = generate_train_image_pairs(faces_resized, labels_dataset)
history = model.fit([images_pair[:, 0], images_pair[:, 1]], labels_pair[:],validation_split=0.1,batch_size=64,epochs=100)

Epoch 1/100
12/12 [==============================] - 12s 164ms/step - loss: 0.6939 - accuracy: 0.5000 - val_loss: 0.6503 - val_accuracy: 0.6000
Epoch 2/100
12/12 [==============================] - 1s 88ms/step - loss: 0.6716 - accuracy: 0.5000 - val_loss: 0.6576 - val_accuracy: 0.6250
Epoch 3/100
12/12 [==============================] - 1s 88ms/step - loss: 0.6734 - accuracy: 0.5000 - val_loss: 0.6629 - val_accuracy: 0.6500
Epoch 4/100
12/12 [==============================] - 1s 87ms/step - loss: 0.6634 - accuracy: 0.5097 - val_loss: 0.6467 - val_accuracy: 0.6625
Epoch 5/100
12/12 [==============================] - 1s 88ms/step - loss: 0.6556 - accuracy: 0.5069 - val_loss: 0.6712 - val_accuracy: 0.7000
Epoch 6/100
12/12 [==============================] - 1s 88ms/step - loss: 0.6631 - accuracy: 0.5819 - val_loss: 0.6322 - val_accuracy: 0.6500
Epoch 7/100
12/12 [==============================] - 1s 88ms/step - loss: 0.6661 - accuracy: 0.5458 - val_loss: 0.6554 - val_accuracy: 0.6625
Epoc

# Testing Pairs Generation

In [12]:
def generate_test_image_pairs(images_dataset, labels_dataset, image):
    unique_labels = np.unique(labels_dataset)
    label_wise_indices = dict()
    for label in unique_labels:
        label_wise_indices.setdefault(label,
                                      [index for index, curr_label in enumerate(labels_dataset) if
                                        label == curr_label])
  
    pair_images = []
    pair_labels = []
    for label, indices_for_label in label_wise_indices.items():
        test_image = images_dataset[np.random.choice(indices_for_label)]
        pair_images.append((image, test_image))
        pair_labels.append(label)
    return np.array(pair_images), np.array(pair_labels)

# Prediction

In [15]:
image = faces_resized[92] # a random image as test image
test_image_pairs, test_label_pairs = generate_test_image_pairs(faces_resized, labels_dataset, image) # produce an array of test image pairs and test label pairs

# for each pair in the test image pair, predict the similarity between the images
for index, pair in enumerate(test_image_pairs):
    pair_image1 = np.expand_dims(pair[0], axis=-1)
    pair_image1 = np.expand_dims(pair_image1, axis=0)
    pair_image2 = np.expand_dims(pair[1], axis=-1)
    pair_image2 = np.expand_dims(pair_image2, axis=0)
    prediction = model.predict([pair_image1, pair_image2])[0][0]

1/1 [==============================] - 0s 14ms/step


Credits :- Girija Shankar Behera

https://medium.com/wicds/face-recognition-using-siamese-networks-84d6f2e54ea4